In [ ]:
import time
from typing import Any, Dict, List, Tuple, Union

import argparse
import joblib
import tensorflow as tf
from termcolor import cprint
import numpy as np
import pickle as pkl
import torch
import torch.nn as nn
import torch.nn.functional as F
import utils.baseline_utils as baseline_utils
from lstm_train_test_det import *

In [4]:
from glob import glob

In [6]:
!ls ../../argoverse

agents_train.npy	       agents_val_rotation.npy	test_obs
agents_train_rotation.npy      agents_val_transi.npy	train
agents_train_small.npy	       argoverse-api.git	train_original
agents_train_small_transi.npy  features			val
agents_train_transi.npy        preprocessing.ipynb	val_original
agents_val.npy		       test


In [7]:
 with open('../../argoverse/agents_train_rotation.npy', 'rb') as f:
    wholetraj = np.load(f)

In [9]:
wholetraj.shape

(205942, 50, 2)

In [12]:
conformal_train = wholetraj[:102971]
conformal_err = wholetraj[102971:]
conformal_train.shape

(102971, 50, 2)

In [13]:
with open('../../argoverse/conformal_train.npy', 'wb') as f:
    np.save(f, conformal_train)
    
with open('../../argoverse/conformal_err.npy', 'wb') as f:
    np.save(f, conformal_err)

In [16]:
device = 'cuda:0'

In [18]:
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)
encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)
encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

model_utils = ModelUtils()

model_path = '../checkpoints/lstm_argo_det/LSTM_rollout30.pth.tar'
epoch, rollout_len, _ = model_utils.load_checkpoint(
            model_path, encoder, decoder, encoder_optimizer,
            decoder_optimizer)

=> no checkpoint found at ../checkpoints/lstm_argo_det/LSTM_rollout30.pth.tar


UnboundLocalError: local variable 'epoch' referenced before assignment

In [14]:
!nvidia-smi

Fri Jan 14 01:08:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 27%   44C    P2    61W / 250W |   6363MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1B:00.0 Off |                  N/A |
| 22%   